In [ ]:
3)

In [6]:
import pandas as pd
import numpy as np

# Descargar los datos de Gaussian Data
url = 'https://raw.githubusercontent.com/asegura4488/Database/main/MetodosComputacionalesReforma/Gaussiano.csv'
data = pd.read_csv(url)
histogram_data = data['x'].values

def prior_uniform_distribution(mean, std_dev):
    value = 0
    if (3 <= mean <= 5) and (0.5 <= std_dev <= 3.5):
        value = 1
    return value

def gaussian_likelihood(mean, std_dev, data):
    likelihood = 1
    for x in data:
        likelihood *= (np.sqrt(2 * np.pi * std_dev**2))**(-1) * np.exp(-(mean - x)**2 / (2 * std_dev**2))
    return likelihood

def calculate_log_posterior(mean, std_dev, data):
    log_posterior = np.log(prior_uniform_distribution(mean, std_dev) * gaussian_likelihood(mean, std_dev, data))
    return log_posterior

def metropolis_algorithm(data):
    num_samples = 2 * 10**4
    mean_current = np.random.uniform(3, 5)
    std_dev_current = np.random.uniform(0.5, 3.5)
    log_posterior_current = calculate_log_posterior(mean_current, std_dev_current, data)
    
    mean_samples = [mean_current]
    std_dev_samples = [std_dev_current]
    
    for _ in range(num_samples):
        mean_proposed = np.random.normal(mean_current, 0.1)
        std_dev_proposed = np.random.normal(std_dev_current, 0.1)
        log_posterior_proposed = calculate_log_posterior(mean_proposed, std_dev_proposed, data)
        
        if log_posterior_proposed > log_posterior_current:
            mean_current = mean_proposed
            std_dev_current = std_dev_proposed
            log_posterior_current = log_posterior_proposed
        else:
            acceptance_prob = np.exp(log_posterior_proposed - log_posterior_current)
            if np.random.uniform() < acceptance_prob:
                mean_current = mean_proposed
                std_dev_current = std_dev_proposed
                log_posterior_current = log_posterior_proposed
                
        mean_samples.append(mean_current)
        std_dev_samples.append(std_dev_current)
    
    return mean_samples, std_dev_samples

mean_samples, std_dev_samples = metropolis_algorithm(histogram_data)

# Estimar el mejor valor de los parámetros del modelo (mean_hat, std_dev_hat)
mean_hat = np.percentile(mean_samples, 50)
std_dev_hat = np.percentile(std_dev_samples, 50)

print("El mejor valor estimado para la media es:", mean_hat)
print("El mejor valor estimado para la desviación estándar es:", std_dev_hat)

# Encontrar los errores sigma+ y sigma- de los parámetros en un intervalo de confianza del CL = 68%
mean_low = np.percentile(mean_samples, 16)
mean_up = np.percentile(mean_samples, 84)
std_dev_low = np.percentile(std_dev_samples, 16)
std_dev_up = np.percentile(std_dev_samples, 84)

print("Los valores de error (sigma+) y (sigma-) para la media en un intervalo de confianza del CL = 68% son:", mean_hat - mean_low, "y", mean_up - mean_hat)
print("Los valores de error (sigma+) y (sigma-) para la desviación estándar en un intervalo de confianza del CL = 68% son:", std_dev_hat - std_dev_low, "y", std_dev_up - std_dev_hat)


<ipython-input-6-e6483030fbe7>:22: RuntimeWarning: divide by zero encountered in log
  log_posterior = np.log(prior_uniform_distribution(mean, std_dev) * gaussian_likelihood(mean, std_dev, data))


El mejor valor estimado para la media es: 4.006037020371882
El mejor valor estimado para la desviación estándar es: 1.8347941975802142
Los valores de error (sigma+) y (sigma-) para la media en un intervalo de confianza del CL = 68% son: 0.1797623657039975 y 0.18392066794940565
Los valores de error (sigma+) y (sigma-) para la desviación estándar en un intervalo de confianza del CL = 68% son: 0.1238500091857686 y 0.1336493521124913


In [8]:
pip install emcee


Note: you may need to restart the kernel to use updated packages.


4)

In [12]:
import pandas as pd
import numpy as np
import emcee

url = 'https://raw.githubusercontent.com/asegura4488/Database/main/MetodosComputacionalesReforma/Gaussiano.csv'
data = pd.read_csv(url)
histogram_data = data['x'].values

def prior_uniform_distribution(mean, std_dev):
    value = 0
    if (3 <= mean <= 5) and (0.5 <= std_dev <= 3.5):
        value = 1
    return value

def gaussian_likelihood(mean, std_dev, data):
    likelihood = 1
    for x in data:
        likelihood *= (np.sqrt(2 * np.pi * std_dev**2))**(-1) * np.exp(-(mean - x)**2 / (2 * std_dev**2))
    return likelihood

def log_prior(params):
    mean, std_dev = params
    return np.log(prior_uniform_distribution(mean, std_dev))

def log_likelihood(params):
    mean, std_dev = params
    return np.log(gaussian_likelihood(mean, std_dev, histogram_data))

def log_posterior(params):
    return log_prior(params) + log_likelihood(params)

ndim = 2
nwalkers = 50
nsteps = 500

pos = np.random.uniform(low=[3, 0.5], high=[5, 3.5], size=(nwalkers, ndim))

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior)

sampler.run_mcmc(pos, nsteps, progress=True)

samples = sampler.get_chain(discard=100, flat=True)

mean_hat = np.percentile(samples[:, 0], 50)
std_dev_hat = np.percentile(samples[:, 1], 50)

mean_low = np.percentile(samples[:, 0], 16)
mean_up = np.percentile(samples[:, 0], 84)
std_dev_low = np.percentile(samples[:, 1], 16)
std_dev_up = np.percentile(samples[:, 1], 84)

print(mean_hat)
print(std_dev_hat)
print(mean_hat - mean_low)
print(mean_up - mean_hat)
print(std_dev_hat - std_dev_low)
print(std_dev_up - std_dev_hat)


  0%|                                                                                          | 0/500 [00:00<?, ?it/s]<ipython-input-12-b06763db62f1>:23: RuntimeWarning: divide by zero encountered in log
  return np.log(prior_uniform_distribution(mean, std_dev))
<ipython-input-12-b06763db62f1>:27: RuntimeWarning: divide by zero encountered in log
  return np.log(gaussian_likelihood(mean, std_dev, histogram_data))
100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:15<00:00, 32.96it/s]

4.022037573211957
1.828896447293159
0.1883332952169905
0.18268307062057332
0.12820216270390627
0.13957937546205001
